In [1]:
 """
        @Author: Alexander Pabel
"""
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim

In [2]:
#Bilder augmentieren fürs Training und die Evaluation
train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
class catDogImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_to_idx = {'cats': 0, 'dogs': 1}
        
        for label in os.listdir(root_dir):
            class_dir = os.path.join(root_dir, label)
            if os.path.isdir(class_dir):
                for img_name in os.listdir(class_dir):
                    self.image_paths.append(os.path.join(class_dir, img_name))
                    self.labels.append(self.class_to_idx[label])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

train_dataset = catDogImageDataset(root_dir='dataset/train', transform=train_transforms)
val_dataset = catDogImageDataset(root_dir='dataset/val', transform=val_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class catDogClassifierCNN(nn.Module):
    def __init__(self):
        super(catDogClassifierCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Dropout-Schichten zur Regularisierung nach den kovolutionalen Schichten um Overfitting zu vermeiden
        self.dropout_conv1 = nn.Dropout(p=0.2)  # Dropout nach der ersten konvolutionalen Schicht
        self.dropout_conv2 = nn.Dropout(p=0.3)  # Dropout nach der zweiten konvolutionalen Schicht
        self.dropout_conv3 = nn.Dropout(p=0.3)  # Dropout nach der dritten konvolutionalen Schicht
        
        self.fc1 = nn.Linear(256 * 8 * 8, 512)
        self.dropout1 = nn.Dropout(p=0.5)  # Erhöhung der dropout rate nach der vollverbundenen Schicht
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout_conv1(x)  # Anwenden des dropouts nach der ersten konvolutionalen Schicht
        
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout_conv2(x)  # Anwenden des dropouts nach der zweiten konvolutionalen Schicht
        
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout_conv3(x)  # Anwenden des dropouts nach der dritten konvolutionalen Schicht
        
        x = self.pool(F.relu(self.conv4(x)))
        
        # Flatten: den mehrdimensionalen Output der konvolutionalen Schicht in ein eindimensionales Array
        # bringen für die vollverbundene Schicht
        x = x.view(-1, 256 * 8 * 8)
        
        # Fully connected layer with dropout
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        
        x = self.fc2(x)
        return x

model = catDogClassifierCNN()

In [ ]:
#Criterion für Fehlerberechnung,Gradientenberechnung und Backpropagation
criterion = nn.CrossEntropyLoss()
#Optimierer
optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=0.001)

num_epochs = 70 

best_val_loss = float('inf')
patience = 10  # Number of epochs to wait before stopping
epochs_without_improvement = 0

#Training des Modells
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement>=4:
            optimizer = optim.Adam(model.parameters(), lr=0.0001,weight_decay=0.01)

    if epochs_without_improvement >= patience:
        print("Early stopping")

        break